<a href="https://colab.research.google.com/github/bountyhunter12/DL/blob/main/age_gender_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [52]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [53]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [54]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [55]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [56]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [57]:
len(age)

23708

In [58]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [59]:
df.shape

(23708, 3)

In [60]:
df.head()

,age,gender,img
0,60,0,60_0_0_20170117173825225.jpg.chip.jpg
1,26,1,26_1_1_20170117155009422.jpg.chip.jpg
2,40,1,40_1_0_20170104204319051.jpg.chip.jpg
3,32,1,32_1_1_20170113012739969.jpg.chip.jpg
4,32,1,32_1_0_20170109134507157.jpg.chip.jpg


In [61]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [62]:
train_df.shape

(20000, 3)

In [63]:
test_df.shape

(3708, 3)

In [89]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import Flatten, Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model


In [90]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [91]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=(200, 200),
    class_mode='raw'  # ✅ Fix here
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=(200, 200),
    class_mode='raw'  # ✅ Fix here too
)


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [92]:
from keras.layers import *

In [93]:
resnet = ResNet50(include_top=False, input_shape=(200, 200, 3), weights='imagenet')
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
age_output = Dense(1, name='age')(x)
gender_output = Dense(1, activation='sigmoid', name='gender')(x)

In [83]:
# resnet = ResNet50(include_top=False, input_shape=(200,200,3))

# resnet.trainable=False

# output = resnet.layers[-1].output

# flatten = Flatten()(output)

# dense1 = Dense(512, activation='relu')(flatten)
# dense2 = Dense(512,activation='relu')(flatten)

# dense3 = Dense(512,activation='relu')(dense1)
# dense4 = Dense(512,activation='relu')(dense2)

# output1 = Dense(1,activation='linear',name='age')(dense3)
# output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [94]:
# Final model
model = Model(inputs=resnet.input, outputs=[age_output, gender_output])

In [95]:
model.compile(
    optimizer='adam',
    loss={'age': 'mae', 'gender': 'binary_crossentropy'},
    metrics={'age': 'mae', 'gender': 'accuracy'},
    loss_weights={'age': 1, 'gender': 99}
)


In [96]:
def multi_output_wrapper(generator):
    for x, y in generator:
        yield x, {'age': y[:, 0], 'gender': y[:, 1]}


In [87]:
# train_gen_split = multi_output_wrapper(train_generator)
# test_gen_split = multi_output_wrapper(test_generator)


In [97]:
train_gen_split = multi_output_wrapper(train_generator)
test_gen_split = multi_output_wrapper(test_generator)

model.fit(
    train_gen_split,
    steps_per_epoch=len(train_generator),
    validation_data=test_gen_split,
    validation_steps=len(test_generator),
    epochs=10
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 323s 418ms/step - age_loss: 18.5197 - age_mae: 18.5197 - gender_accuracy: 0.7242 - gender_loss: 0.6120 - loss: 79.1070 - val_age_loss: 24.0468 - val_age_mae: 24.0466 - val_gender_accuracy: 0.4752 - val_gender_loss: 0.7136 - val_loss: 94.6968
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 256s 411ms/step - age_loss: 11.4505 - age_mae: 11.4505 - gender_accuracy: 0.8367 - gender_loss: 0.3548 - loss: 46.5740 - val_age_loss: 14.4577 - val_age_mae: 14.4577 - val_gender_accuracy: 0.8068 - val_gender_loss: 0.4184 - val_loss: 55.8834
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 252s 404ms/step - age_loss: 10.1607 - age_mae: 10.1607 - gender_accuracy: 0.8584 - gender_loss: 0.3155 - loss: 41.3921 - val_age_loss: 17.2161 - val_age_mae: 17.2161 - val_gender_accuracy: 0.7883 - val_gender_loss: 0.4000 - val_loss: 56.8200
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 255s 408ms/step - age_loss: 9.5094 - age_mae: 9.5094 - gender_accuracy: 0.8653 - gender_loss: 0.2976 - loss: 38.97